## Importando bibliotecas

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

## Carregando dataset

In [2]:
resenha = pd.read_csv('imdb-reviews-pt-br.csv')
resenha.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [3]:
linhas, colunas = resenha.shape
print(f'Linhas: {linhas}\nColunas: {colunas}')

Linhas: 49459
Colunas: 4


## Conhecendo os dados

#### Exemplo de texto negativo

In [4]:
print(f'Texto negativo:\n{resenha.text_pt[10]}')

Texto negativo:
Cage interpreta um bêbado e é elogiado pela crítica. Elizabeth Shue Na verdade, tem que fazer um amor com a parte mais desprezível e superestimada de estrume em Hollywood. Eu literalmente vomitei enquanto assistia a esse filme. Claro que tive gripe, mas isso não significa que este filme não tenha contribuído para o vômito no kamode. Por que Nick Cage não pode tocar em algo que ele pode realmente fazer como um ator ruim? Nick Cage, que é brilhante em um papel de mau ator. Parreira ninguém poderia fazer melhor. A busca começa por Nicks contrato com Lúcifer ou Lou Cipher de "Night Train To Terror".


#### Exemplo de texto positivo

In [5]:
print(f'Texto positivo:\n{resenha.text_pt[49002]}')

Texto positivo:
Crescendo em Nova York no final dos anos 80 e início dos anos 90, posso dizer pessoalmente que este é um dos documentários mais importantes feitos para cobrir esse lugar neste período de tempo. Não Madonna não veio com a idéia de Voguing, mas é de onde ela tirou! Em vez de combater a violência uns dos outros ou em brigas de gato, o voguing permitia que as pessoas "lutassem" dentro dos confins de tudo, menos que tocassem umas às outras, o que justificaria uma desqualificação automática. Vendo este tipo de extraordinariamente talentosas / bem orquestradas "jogadas" nos clubes foi nada menos do que espetacular e todos os grandes nomes de antigamente estão aqui ... Pepper La Beija, Paris Duprée, Xtragavaganza, etc. ..tudo comemorado nos gostos de peças de época como a música de Malcom McLarens "Deep in Vogue" ... não importava quem você era, ou de onde você era porque quando você passava por aquelas portas nesse "reino mágico" de De certa forma, você se tornou parte de algo

#### Contagem dos valores e substituição

In [6]:
# Quantidade de textos positivos e negativos.
resenha['sentiment'].value_counts()

sentiment
neg    24765
pos    24694
Name: count, dtype: int64

In [7]:
# Criando uma nova coluna para classificar os sentimentos em 0 e 1.
resenha['classificacao'] = resenha['sentiment'].replace(['neg', 'pos'], [0, 1])

## Criando o modelo

### Separando os dados de treino e teste

In [8]:
treino, teste, classe_treino, classe_teste = train_test_split(resenha['text_pt'], resenha['sentiment'], random_state = 42)

In [9]:
#regressao_logistica = LogisticRegression()
#regressao_logistica.fit(treino, classe_treino)
#acuracia = regressao_logistica.score(teste, classe_teste)
#print(f'Acurácia: {acuracia}')